In [88]:
import numpy as np
import pandas as pd

%load_ext autoreload
%autoreload 2
#%matplotlib inline

pindir="../pin_experiments"
querydir="query1_cores16_frate100000_6000000_fbuff-1_itr1_ondemanddvfs1_source16_mapper16_sink16_repeat0"
configs = ["base", "C1", "C2", "C3", "C4", "C5"]

df_dict = {
    "config" : [],
    "pkgPower" : [],
    "pkgPowerstd" : [],
    "dramPower" : [],
    "dramPowerstd" : [],
    "SourceOut" : [],
    "POLL" : [],
    "C1" : [],
    "C1E" : [],
    "C3" : [],
    "C6" : [],
    "cpu_user" : [],
    "cpu0_user": [],
    "cpu1_user": [],
    "cpu2_user": [],
    "cpu3_user": [],
    "cpu4_user": [],
    "cpu5_user": [],
    "cpu6_user": [],
    "cpu7_user": [],
    "cpu8_user": [],
    "cpu9_user": [],
    "cpu10_user": [],
    "cpu11_user": [],
    "cpu12_user": [],
    "cpu13_user": [],
    "cpu14_user": [],
    "cpu15_user": [],
    "cpu_idle" : [],
    "cpu0_idle": [],
    "cpu1_idle": [],
    "cpu2_idle": [],
    "cpu3_idle": [],
    "cpu4_idle": [],
    "cpu5_idle": [],
    "cpu6_idle": [],
    "cpu7_idle": [],
    "cpu8_idle": [],
    "cpu9_idle": [],
    "cpu10_idle": [],
    "cpu11_idle": [],
    "cpu12_idle": [],
    "cpu13_idle": [],
    "cpu14_idle": [],
    "cpu15_idle": []
}

def resetdict():
    global df_dict
    
    df_dict = {
    "config" : [],
    "pkgPower" : [],
    "pkgPowerstd" : [],
    "dramPower" : [],
    "dramPowerstd" : [],
    "SourceOut" : [],
    "POLL" : [],
    "C1" : [],
    "C1E" : [],
    "C3" : [],
    "C6" : [],
    "cpu_user" : [],
    "cpu0_user": [],
    "cpu1_user": [],
    "cpu2_user": [],
    "cpu3_user": [],
    "cpu4_user": [],
    "cpu5_user": [],
    "cpu6_user": [],
    "cpu7_user": [],
    "cpu8_user": [],
    "cpu9_user": [],
    "cpu10_user": [],
    "cpu11_user": [],
    "cpu12_user": [],
    "cpu13_user": [],
    "cpu14_user": [],
    "cpu15_user": [],
    "cpu_idle" : [],
    "cpu0_idle": [],
    "cpu1_idle": [],
    "cpu2_idle": [],
    "cpu3_idle": [],
    "cpu4_idle": [],
    "cpu5_idle": [],
    "cpu6_idle": [],
    "cpu7_idle": [],
    "cpu8_idle": [],
    "cpu9_idle": [],
    "cpu10_idle": [],
    "cpu11_idle": [],
    "cpu12_idle": [],
    "cpu13_idle": [],
    "cpu14_idle": [],
    "cpu15_idle": []
}

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [90]:
resetdict()

for c in configs:
    fname = f"{pindir}/{querydir}/{c}"

    df_dict['config'].append(c)
    
    # Power
    pkgs=[]
    dram=[]
    with open(f"{fname}/rapl.log") as f:
        lines = f.readlines()
    for line in lines:
        pkgs.append(float(line.strip().split(' ')[0]))
        dram.append(float(line.strip().split(' ')[1]))
    print(f"CPU pkg: {np.mean(np.array(pkgs))} {np.std(np.array(pkgs))}")
    print(f"DRAM pkg: {np.mean(np.array(dram))} {np.std(np.array(dram))}")
    df_dict['pkgPower'].append(np.mean(np.array(pkgs)))
    df_dict['pkgPowerstd'].append(np.std(np.array(pkgs)))
    df_dict['dramPower'].append(np.mean(np.array(dram)))
    df_dict['dramPowerstd'].append(np.std(np.array(dram)))

    # Source records out
    kwlist={'numRecordsOutPerSecond':[]}

    with open(f"{fname}/Source.0") as f:
        lines = f.read()
    for lc in lines.split('; '):
        for kw in kwlist.keys():
            if(kw in lc):
                ldict=eval(lc.replace('[','').replace(']',''))
                sourceout = float(ldict['value'])
    print(f"Source out: {sourceout}")
    df_dict['SourceOut'].append(sourceout)
    
    # idle stats
    statsSTART = pd.read_csv(f"{fname}/statsSTART.csv", names=['POLL', 'C1', 'C1E', 'C3', 'C6', 
                                                       'RXP', 'RXB', 'TXP', 'TXB', 'ERXP', 'ERXB', 'ETXP', 'ETXB'])
    statsEND = pd.read_csv(f"{fname}/statsEND.csv", names=['POLL', 'C1', 'C1E', 'C3', 'C6', 
                                                       'RXP', 'RXB', 'TXP', 'TXB', 'ERXP', 'ERXB', 'ETXP', 'ETXB'])
    statsdf = pd.concat([statsSTART, statsEND])
    statsdf = statsdf.diff()
    statsdf.dropna(inplace=True)
    df_dict['POLL'].append(statsdf['POLL'].values[0])
    df_dict['C1'].append(statsdf['C1'].values[0])
    df_dict['C1E'].append(statsdf['C1E'].values[0])
    df_dict['C3'].append(statsdf['C3'].values[0])
    df_dict['C6'].append(statsdf['C6'].values[0])
    #print(statsdf[['POLL', 'C1', 'C3', 'C1E', 'C6']])

    # /proc/stat
    with open(f"{fname}/procstatsSTART.csv") as f:
        lines = f.readlines()

    procstats={}
    for line in lines:
        if 'cpu' in line:
            stats = line.strip().split(' ')
            #print(stats)
            tmp = []
            for i in range(1, len(stats)):
                if len(stats[i]) > 0:
                    tmp.append(int(stats[i]))
            procstats[stats[0]] = tmp
            #print(stats[0], len(tmp))
    
    with open(f"{fname}/procstatsEND.csv") as f:
        lines = f.readlines()
    for line in lines:
        if 'cpu' in line:
            stats = line.strip().split(' ')
            tmp = []
            #print(stats)
            for i in range(1, len(stats)):
                if len(stats[i]) > 0:
                    tmp.append(int(stats[i]))
            for i in range(0, len(tmp)):
                procstats[stats[0]][i] = tmp[i] - procstats[stats[0]][i]
    #print(procstats)  

    for k, v in procstats.items():
        df_dict[f"{k}_user"].append(v[0])
        df_dict[f"{k}_idle"].append(v[3])

dd1 = pd.DataFrame(df_dict)
print(dd1.columns)
dd1[['config', 'pkgPower', 'pkgPowerstd', 'dramPower', 'dramPowerstd', 'SourceOut', 'POLL', 'C1', 'C1E', 'C3', 'C6', 'cpu_user', 'cpu_idle']]

CPU pkg: 51.04497229914529 0.47817455012264715
DRAM pkg: 10.841677726495725 0.070733063652622
Source out: 100000.0
CPU pkg: 49.709064316239306 0.6070365171875638
DRAM pkg: 10.682762752136751 0.14127786894293887
Source out: 100000.0
CPU pkg: 50.33084403418804 0.6590846256961944
DRAM pkg: 10.740796290598288 0.10778023030087858
Source out: 100000.0
CPU pkg: 49.56189821367521 0.7073599461381099
DRAM pkg: 10.470422957264956 0.14112467671896492
Source out: 100000.0
CPU pkg: 51.535734418803415 0.7762051876274059
DRAM pkg: 10.562458931623933 0.15066978589888502
Source out: 100000.0
CPU pkg: 52.26167924786324 0.6169349114196941
DRAM pkg: 10.660945948717949 0.10092776876430476
Source out: 100000.0
Index(['config', 'pkgPower', 'pkgPowerstd', 'dramPower', 'dramPowerstd',
       'SourceOut', 'POLL', 'C1', 'C1E', 'C3', 'C6', 'cpu_user', 'cpu0_user',
       'cpu1_user', 'cpu2_user', 'cpu3_user', 'cpu4_user', 'cpu5_user',
       'cpu6_user', 'cpu7_user', 'cpu8_user', 'cpu9_user', 'cpu10_user',
       

,config,pkgPower,pkgPowerstd,dramPower,dramPowerstd,SourceOut,POLL,C1,C1E,C3,C6,cpu_user,cpu_idle
0,base,51.044972,0.478175,10.841678,0.070733,100000.0,435.0,688222.0,52530.0,0.0,248992.0,41686,146579
1,C1,49.709064,0.607037,10.682763,0.141278,100000.0,172.0,152983.0,33600.0,0.0,143947.0,41442,146842
2,C2,50.330844,0.659085,10.740796,0.107780,100000.0,119.0,148870.0,30253.0,0.0,142878.0,41447,146779
3,C3,49.561898,0.707360,10.470423,0.141125,100000.0,421.0,715578.0,46908.0,0.0,196903.0,41533,146968
4,C4,51.535734,0.776205,10.562459,0.150670,100000.0,671.0,1119410.0,29674.0,0.0,163613.0,41810,145921
5,C5,52.261679,0.616935,10.660946,0.100928,100000.0,683.0,1134731.0,29680.0,0.0,167907.0,41586,146362
